In [1]:
# Задаем дату до которой теоритически будем прогнозировать
end_future_date = '2024-12-31 23:55:00+00:00'

In [2]:
import pandas as pd

# Читаем данные на которых будем обучать модель
df_Pl = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRKJzXJlQSNew0dxrQ8mFQMhBv_4owvfsF2If1b-rmxMZkR5gabHC4OiaSwt8Ul1Omc8taR27UohSeg/pub?gid=429486365&single=true&output=csv')


In [3]:
# Узнаем последнию известную дату. Нужна для создания промежутка значений будущих дат
end_know_date = df_Pl['time'].iloc[-1]

end_know_date_index = df_Pl[df_Pl['time'] == end_know_date].index[0] + 1


In [4]:
import numpy as np

# Создаем общий датасет с известными и будущими данными нужно для корректного общего нормирования данных

# Добавляем 5 минут чтобы последняя известная дата не попала в список для генерации будущих дат

end_know_date = pd.to_datetime(end_know_date)

# Добавление 5 минут
end_know_date = end_know_date + pd.Timedelta(minutes=5)

# Преобразование обратно в строку
end_know_date = end_know_date.strftime('%Y-%m-%d %H:%M:%S%z')

future_dates = pd.date_range(start=end_know_date, end=end_future_date, freq='5T')

future_df = pd.DataFrame({'time': future_dates, 'P_l': np.nan})

avg_pl = df_Pl['P_l'].mean()

# Заполняем Nan если они есть в датафрейме с известными данными
df_Pl['P_l'].fillna(avg_pl, inplace=True)

# Создаем общий датаферйм с известными и будущими датами для корректной нормировки данных относительно существующих и будущих дат
df_Pl_sacler = pd.concat([df_Pl, future_df], ignore_index=True)


In [5]:
# Функция нормировки дат относительно всех дат(известных и которых хотим прогнощировать)

def normalized_df(df, scaler):
# Преобразование времени в формат datetime
  df['time'] = pd.to_datetime(df['time'])
  # Создание новых столбцов
  df['year'] = df['time'].dt.year
  df['month'] = df['time'].dt.month
  df['day'] = df['time'].dt.day
  df['hour'] = df['time'].dt.hour
  df['minute'] = df['time'].dt.minute

  # Удаление исходной колонки времени
  df = df.drop('time', axis=1)

  # Нормализация данных
  df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
  return df_normalized

# Обратное масштабирование данных времени
def restored_date(df_normalized, scaler):
  df_restored = pd.DataFrame(scaler.inverse_transform(df_normalized), columns=df_normalized.columns)
  df_restored['time'] = pd.to_datetime(df_restored[['year', 'month', 'day', 'hour', 'minute']])
  df_restored = df_restored.drop(['year', 'month', 'day', 'hour', 'minute'], axis=1)
  return df_restored

In [9]:
from sklearn.preprocessing import MinMaxScaler

# Нормализация всех данных существующих и будущих
scaler = MinMaxScaler(feature_range=(0, 1))

df_Pl_normalized_all_dates = normalized_df(df_Pl_sacler, scaler)


In [17]:
# Данные для обучения отнормированные относительно всего промежутка(известных дат и дат для прогноза)

df_Pl_normalized_initial_dates = df_Pl_normalized_all_dates.loc[: end_know_date_index-1]


In [18]:
df_Pl_normalized_future_dates = df_Pl_normalized_all_dates.loc[end_know_date_index :]


In [26]:
# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day', 'hour', 'minute']].values
y_initial = df_Pl_normalized_initial_dates['P_l'].values


In [29]:

# gреобразование в 5 мерный массив
X_initial_train = X_initial.reshape(-1, 1, 5)


In [31]:
X_initial_train

array([[[0.        , 0.45454545, 0.3       , 0.69565217, 0.90909091]],

       [[0.        , 0.45454545, 0.3       , 0.69565217, 1.        ]],

       [[0.        , 0.45454545, 0.3       , 0.73913043, 0.        ]],

       ...,

       [[0.875     , 0.72727273, 0.33333333, 0.60869565, 0.90909091]],

       [[0.875     , 0.72727273, 0.33333333, 0.60869565, 1.        ]],

       [[0.875     , 0.72727273, 0.33333333, 0.65217391, 0.        ]]])

In [33]:
from sklearn.model_selection import train_test_split

# Данные для обучения
X_initial = df_Pl_normalized_initial_dates[['year', 'month', 'day', 'hour', 'minute']].values
y_initial = df_Pl_normalized_initial_dates['P_l'].values

# Преобразование в 5-мерный массив
X_initial_train = X_initial.reshape(-1, 1, 5)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_initial_train, y_initial, test_size=0.01, random_state=42)


In [47]:
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, concatenate
from tensorflow.keras.models import Model

print(X_train.shape)
# Создание входов
input_cnn = Input(shape=(1, 5), name='input_cnn')
input_lstm = Input(shape=(1, 5), name='input_lstm')
print(input_cnn)
# Сверточный слой для признака P_l
conv1 = Conv1D(filters=32, kernel_size=1, activation='relu')(input_cnn)  # изменено kernel_size на 1
pool1 = MaxPooling1D(pool_size=1)(conv1)
flat1 = Flatten()(pool1)

# LSTM слой для времени
lstm = LSTM(units=50, activation='relu')(input_lstm)

# Объединение выходов сверточного слоя и выходов для времени
merged = concatenate([flat1, lstm])

dense = Dense(units=50, activation='relu')(merged)

output = Dense(units=1, activation='linear')(dense)

model = Model(inputs=[input_cnn, input_lstm], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')

# Обучение модели
epochs = 10
batch_size = 512
model.fit([X_train, X_train], y_train, epochs=epochs, batch_size=batch_size)


(755262, 1, 5)
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 5), dtype=tf.float32, name='input_cnn'), name='input_cnn', description="created by layer 'input_cnn'")
Epoch 1/10
1476/1476 [==============================] - 4s 2ms/step - loss: 0.0029
Epoch 2/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0020
Epoch 3/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0018
Epoch 4/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0016
Epoch 5/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0015
Epoch 6/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0015
Epoch 7/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0014
Epoch 8/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0014
Epoch 9/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0014
Epoch 10/10
1476/1476 [==============================] - 3s 2ms/step - loss: 0.0013
